## 데이터 불러오기

In [ ]:
import pandas as pd

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

## 데이터 전처리

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(f"[{string.punctuation}]", " ", text)
    
    # 소문자로 변환
    text = text.lower()
    
    # stopwords 제거
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    return text

train_df["preprocessed_text"] = train_df["text"].apply(preprocess_text)
test_df["preprocessed_text"] = test_df["text"].apply(preprocess_text)


# Dataset 클래스

In [ ]:
!pip install transformers

In [ ]:
from torch.utils.data import Dataset
from transformers import BertTokenizer

class NewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, is_test=False):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test
        self.encoded_dict = {}
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if idx not in self.encoded_dict:
            text = self.dataframe.loc[idx, "preprocessed_text"]
            self.encoded_dict[idx] = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt',
                return_attention_mask=True
            )
            
        item = {
            'input_ids': self.encoded_dict[idx]['input_ids'].squeeze(0),
            'attention_mask': self.encoded_dict[idx]['attention_mask'].squeeze(0)
        }
        
        if not self.is_test:
            item['labels'] = self.dataframe.loc[idx, 'label']
            
        return item


## Dataset, DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizer

# Set random seeds
random_seed = 42
torch.manual_seed(random_seed)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Create the datasets
train_val_dataset = NewsDataset(train_df, tokenizer)
test_dataset = NewsDataset(test_df, tokenizer, is_test=True)

# Split the train dataset into train and validation sets
train_size = int(0.8 * len(train_val_dataset))
valid_size = len(train_val_dataset) - train_size

train_dataset, valid_dataset = random_split(train_val_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(random_seed))

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


## Model

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=8)


## criterion, optimizer, metric

In [ ]:
from transformers import AdamW
from torch import nn
from sklearn.metrics import f1_score

# Criterion
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=2e-2)

# Metric
def macro_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='macro')
